In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Setting up paths

In [2]:
lab_path = r"D:\Accesos directos\Trabajo\World Bank\Peru Amag\peru-amag-stats\Data"
lab_data_name = "\Clean_Full_Data12.dta"

In [3]:
lab_data = pd.read_stata(lab_path + lab_data_name)

In [4]:
# renaming gender column
lab_data.columns = lab_data.columns.str.replace('GÃ©nero', 'Gender')

In [5]:
judges_characteristics = ["Age_rounded", "Cargo", "Género"]

# Preparing data for OLS

## Generating dummies

In [6]:
position_dummies = pd.get_dummies(lab_data["Cargo"]).drop(["", "ASIS"], axis=1)
gender_dummies = pd.get_dummies(lab_data["Gender"]).drop(["", "Masculino"], axis=1)
course_dummies = pd.get_dummies(lab_data["Curso"]).drop([""], axis=1)
course_dummies.columns = ["Control", "Interpretacion", "Jurisprudencia", "Razonamiento", "Virtudes", "Etica"]
course_dummies = course_dummies.drop(columns = ["Control"])

## Creating outcomes

In [7]:
# merging data with dummies
lab_reg_data = pd.concat([lab_data, position_dummies, gender_dummies, course_dummies], axis=1)

# squaring age
lab_reg_data["Age_squared"] = lab_reg_data["Age_rounded"]**2

# generating outcomes
lab_reg_data["iat_score_change"] = lab_reg_data["en_iat_score"] - lab_reg_data["bs_iat_score"]

## OLS regression

In [8]:
covariates = ["Age_rounded", "Age_squared", "bs_iat_score"] + list(position_dummies.columns) + list(gender_dummies.columns) + list(course_dummies.columns)

In [9]:
score_change_reg_output = sm.OLS(lab_reg_data["iat_score_change"], sm.add_constant(lab_reg_data[covariates]), missing="drop").fit()
en_score_reg_output = sm.OLS(lab_reg_data["en_iat_score"], sm.add_constant(lab_reg_data[covariates]), missing="drop").fit()

In [10]:
en_score_reg_output.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           en_iat_score   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     1.439
Date:                Sat, 03 Sep 2022   Prob (F-statistic):              0.160
Time:                        14:25:25   Log-Likelihood:                -14.574
No. Observations:                 120   AIC:                             55.15
Df Residuals:                     107   BIC:                             91.39
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.2378      0.594     -0.400      0.690      -1.416       0.941
Age_rounded        0.0115      0.030      0.388      0.699      -0.047       0.071
Age_squared       -0.0002      0.000     -0.493      0.623      -0.001       0.001
bs_iat_score       0.0417      0.086      0.485      0.628      -0.128       0.212
AUX                0.0385      0.089      0.435      0.665      -0.137       0.214
FISCAL            -0.1294      0.090     -1.438      0.153      -0.308       0.049
JUEZ              -0.0663      0.098     -0.678      0.499      -0.260       0.128
Femenino           0.0054      0.058      0.094      0.926      -0.110       0.121
Interpretacion    -0.0691      0.095     -0.727      0.469      -0.258       0.119
Jurisprudencia    -0.0145      0.087     -0.165      0.869      -0.188       0.159
Razonamiento      -0.1161      0.087     -1.329      0.187      -0.289       0.057
Virtudes          -0.1171      0.113     -1.036      0.303      -0.341       0.107
Etica              0.1496      0.109      1.378      0.171      -0.066       0.365
==============================================================================
Omnibus:                        0.419   Durbin-Watson:                   2.098
Prob(Omnibus):                  0.811   Jarque-Bera (JB):                0.299
Skew:                           0.122   Prob(JB):                        0.861
Kurtosis:                       3.000   Cond. No.                     4.09e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.09e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Substracting the variables that have a significant impact over our outcome

- For the IAT Score Change, only the Baseline IAT Score is significant
- For the IAT Endline Score, there are no significant covariates

In [11]:
lab_reg_data["iat_sc_residuzalized"] = lab_reg_data["iat_score_change"] - score_change_reg_output.params["bs_iat_score"]*lab_reg_data["bs_iat_score"]

# Preparing Data for A1 and A2 calculations

## Baseline

### Filtering the data for calculating A1 and A2

In [12]:
a1_data_bs = lab_reg_data[(lab_reg_data["bs_iat_show_feedback"]==1) & ((lab_reg_data["bs_iat_feedback_level"]>=1) & (lab_reg_data["bs_iat_feedback_level"]<=5))]
a2_data_bs = lab_reg_data[((lab_reg_data["bs_iat_feedback_level"]>=1) & (lab_reg_data["bs_iat_feedback_level"]<=5))]

In [13]:
a1_data_bs[["iat_sc_residuzalized", "en_iat_score"]]

,iat_sc_residuzalized,en_iat_score
114,-0.245652,-0.247543
119,NaN,NaN
162,NaN,NaN
212,0.007387,0.013156
227,-0.439276,-0.440208
252,NaN,NaN
297,NaN,NaN
357,-0.571590,-0.590289
434,-0.265792,-0.249609
875,NaN,NaN


In [14]:
a2_data_bs[["iat_sc_residuzalized", "en_iat_score"]]

,iat_sc_residuzalized,en_iat_score
114,-0.245652,-0.247543
119,NaN,NaN
141,-0.697558,-0.719931
162,NaN,NaN
212,0.007387,0.013156
223,-0.304609,-0.318011
227,-0.439276,-0.440208
250,NaN,NaN
252,NaN,NaN
269,-0.223249,-0.241580


## Endline

### Filtering the data for calculating A1 and A2

In [15]:
a1_data_en = lab_reg_data[(lab_reg_data["en_iat_show_feedback"]==1) & ((lab_reg_data["en_iat_feedback_level"]>=1) & (lab_reg_data["en_iat_feedback_level"]<=5))]
a2_data_en = lab_reg_data[((lab_reg_data["en_iat_feedback_level"]>=1) & (lab_reg_data["en_iat_feedback_level"]<=5))]

### Displaying the data

In [16]:
a1_data_en[["iat_sc_residuzalized", "en_iat_score"]]

,iat_sc_residuzalized,en_iat_score
35,-0.172974,-0.176231
141,-0.697558,-0.719931
211,0.202687,0.205634
339,NaN,-0.242300
370,-0.228460,-0.237444
373,0.305125,0.289761
440,NaN,-0.600998


In [17]:
a2_data_en[["iat_sc_residuzalized", "en_iat_score"]]

,iat_sc_residuzalized,en_iat_score
23,NaN,-0.146068
35,-0.172974,-0.176231
107,NaN,-0.079115
126,NaN,0.386953
141,-0.697558,-0.719931
204,-0.421948,-0.427461
209,NaN,-0.284043
211,0.202687,0.205634
254,NaN,-0.399898
275,-0.261897,-0.285770


# IAT Algebra for E2

## Baseline

### Obtaining A1 and A2

In [18]:
a1_iat_sc_bs = a1_data_bs["iat_sc_residuzalized"].mean() # iat score change
a1_iat_bs = a1_data_bs["en_iat_score"].mean() # iat score endline

a2_iat_sc_bs = a2_data_bs["iat_sc_residuzalized"].mean() # iat score change
a2_iat_bs = a2_data_bs["en_iat_score"].mean() # iat score endline

### Obtaining probabilities

In [19]:
freq_iat_score_bs = a2_data_bs["bs_iat_feedback_level"].value_counts()
freq_iat_score_bs

5.0    20
1.0    12
2.0     3
3.0     2
4.0     2
Name: bs_iat_feedback_level, dtype: int64

In [20]:
total_participants_bs_1_5 = freq_iat_score_bs.sum() 
p_bs = (12/total_participants_bs_1_5)*0.1 \
    + (3/total_participants_bs_1_5)*0.2 \
    + (2/total_participants_bs_1_5)*0.3 \
    + (2/total_participants_bs_1_5)*0.4 \
    + (20/total_participants_bs_1_5)*0.5 

In [21]:
p_bs

0.3384615384615384

### Calculating E2

In [22]:
e2_iat_sc_bs = (a2_iat_sc_bs - p_bs*a1_iat_sc_bs)/(1-p_bs)
e2_iat_bs = (a2_iat_bs - p_bs*a1_iat_bs)/(1-p_bs)

In [23]:
e2_iat_sc_bs

-0.2964768058992315

In [24]:
e2_iat_bs

-0.31052600679442627

## Endline

### Obtaining A1 and A2

In [25]:
a1_iat_sc_en = a1_data_en["iat_sc_residuzalized"].mean() # iat score change
a1_iat_en = a1_data_en["en_iat_score"].mean() # iat score endline

a2_iat_sc_en = a2_data_en["iat_sc_residuzalized"].mean() # iat score change
a2_iat_en = a2_data_en["en_iat_score"].mean() # iat score endline

### Obtaining probabilities

In [26]:
freq_iat_score_en = a2_data_en["en_iat_feedback_level"].value_counts()
freq_iat_score_en

1.0    7
5.0    6
4.0    3
3.0    2
2.0    1
Name: en_iat_feedback_level, dtype: int64

In [28]:
total_participants_en_1_5 = freq_iat_score_en.sum() 
p_en = (7/total_participants_en_1_5)*0.1 \
    + (1/total_participants_en_1_5)*0.2 \
    + (2/total_participants_en_1_5)*0.3 \
    + (3/total_participants_en_1_5)*0.4 \
    + (6/total_participants_en_1_5)*0.5 

In [29]:
p_en

0.3

### Calculating E2

In [30]:
e2_iat_sc_en = (a2_iat_sc_en - p_en*a1_iat_sc_en)/(1-p_en)
e2_iat_en = (a2_iat_en - p_en*a1_iat_en)/(1-p_en)

In [31]:
e2_iat_sc_en

-0.18215627154310238

In [32]:
e2_iat_en

-0.18287389752656125